In [34]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os, re
from sklearn import metrics, preprocessing
from scipy import signal

In [35]:
train_df = pd.read_csv('phase_training.csv', sep='\t')

In [36]:
titan_df = train_df[train_df["names"] == "Titanium"]
titan_df = titan_df.append(train_df.loc[40])

In [37]:
train_df = train_df.drop([36,37,38,39,40])

In [38]:
train_df

,names,peak_1,peak_2,peak_3,peak_4,peak_5,peak_6,peak_7,peak_8
0,Aluminium,4.277112,4.938784,6.984494,8.190048,8.554225,9.877568,10.763834,11.043452
1,Aluminium,4.285717,4.948719,6.998547,8.206522,8.571433,9.897433,10.785485,11.065668
2,Aluminium,4.285875,4.948903,6.998807,8.206825,8.571749,9.897805,10.785881,11.066084
3,Aluminium,4.286619,4.949760,7.000017,8.208246,8.573234,9.899520,10.787754,11.067995
4,Aluminium,4.277112,4.938784,6.984494,8.190048,8.554225,9.877568,10.763834,11.043452
5,Aluminium,4.285717,4.948719,6.998547,8.206522,8.571433,9.897433,10.785485,11.065668
6,Aluminium,4.285875,4.948903,6.998807,8.206825,8.571749,9.897805,10.785881,11.066084
7,Aluminium,4.286619,4.949760,7.000017,8.208246,8.573234,9.899520,10.787754,11.067995
8,Aluminium,4.280897,4.943154,6.990675,8.197292,8.561790,9.886307,10.773354,11.053229
9,Iron,4.885897,5.641749,7.978636,9.355780,9.771799,11.283498,12.295903,12.615336


In [40]:
train_df["a"] = [4.04958, 4.04145, 4.0413, 4.0406, 4.04958, 4.04145, 4.0413, 4.0406, 4.046, 2.898, 3.539, 3.533, 3.527, 3.521, 3.514, 3.509, 3.504, 3.498, 3.494, 3.490, 3.485, 3.480, 3.477, 3.473, 3.468, 3.595, 3.581, 3.569, 3.555, 3.544, 3.531, 3.519, 3.507, 3.495 , 3.484, 3.475]

In [41]:
titan_df["a"] = [2.950, 2.97, 2.97, 2.950, 3.232  ]

In [43]:
titan_df["c"] = [4.686,4.72 , 4.72, 4.686, 5.147]

In [44]:
augmentation = []
for i in range(len(train_df)):
    
    ind = train_df.iloc[[i],]
    ind=ind.reindex(ind.index.repeat(100))
    ind = ind.reset_index(drop=True)
    
    for j in range(len(ind)):
        
        data = ind.iloc[j][["peak_1", "peak_2", "peak_3", "peak_4","peak_5", "peak_6", "peak_7", "peak_8"]].values
        cell_param = ind.iloc[j]["a"]
        
        param = []
        delta_a = np.random.randint(1,150)*0.001
        p = np.random.randint(2)
        if p > 0:
            param = cell_param + delta_a
        else:
            param = cell_param - delta_a
                    
        peaks = []
        a_prop = delta_a / cell_param
        for k in range(len(data)):
            
            delta_d = data[k] * a_prop
            
            if p > 0:
                peaks.append(delta_d + data[k])
            else:
                peaks.append(data[k] - delta_d)
                        
        ind.loc[j,["peak_1", "peak_2", "peak_3", "peak_4","peak_5", "peak_6", "peak_7", "peak_8"]] = peaks
        ind.loc[j,["a"]] = param
    augmentation.append(ind)

In [45]:
len(augmentation)

36

In [46]:
aug_df = pd.concat(augmentation, ignore_index=True)

In [47]:
train_data = pd.concat([aug_df,train_df], ignore_index=True)

In [48]:
train_data

,names,peak_1,peak_2,peak_3,peak_4,peak_5,peak_6,peak_7,peak_8,a
0,Aluminium,4.127134,4.765604,6.739580,7.902861,8.254268,9.531207,10.386396,10.656209,3.90758
1,Aluminium,4.323585,4.992445,7.060383,8.279036,8.647169,9.984891,10.880786,11.163442,4.09358
2,Aluminium,4.176775,4.822924,6.820643,7.997916,8.353549,9.645847,10.511323,10.784381,3.95458
3,Aluminium,4.349989,5.022935,7.103501,8.329597,8.699979,10.045869,10.947237,11.231619,4.11858
4,Aluminium,4.291899,4.955858,7.008640,8.218363,8.583798,9.911716,10.801046,11.081630,4.06358
...,...,...,...,...,...,...,...,...,...,...
3631,Iron,4.921996,5.683433,8.037590,9.424911,9.843992,11.366866,12.386754,12.708547,3.51900
3632,Iron,4.938840,5.702880,8.065088,9.457159,9.877675,11.405760,12.429138,12.752028,3.50700
3633,Iron,4.955797,5.722461,8.092782,9.489629,9.911589,11.444921,12.471814,12.795807,3.49500
3634,Iron,4.971444,5.740528,8.118333,9.519585,9.942888,11.481056,12.511198,12.836212,3.48400


In [49]:
train_data.to_csv("training_dataset_cubic_phase.csv", sep='\t', index=False)